In [17]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

# Base URL for the UCSD newspaper archives
base_url = "https://library.ucsd.edu/dc/"

# Load the subpage links from the subpage_links.json file
with open("subpage_links.json", "r") as f:
    subpage_links = json.load(f)

# List to store all the PDF links
pdf_links = []

# Loop through all the subpages and scrape the PDF links
for subpage_link in subpage_links:
    # Make a GET request to the subpage URL
    response = requests.get(subpage_link)

    # Create a BeautifulSoup object from the response HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the PDF links on the page
    pdf_links_on_page = soup.find_all('a', class_='dams-search-thumbnail-link')

    # Loop through all the PDF links and extract the PDF URL
    for pdf_link_on_page in pdf_links_on_page:
        try:
            # Extract the relative URL for the PDF
            pdf_url = pdf_link_on_page['href']

            pdf_url = pdf_url.replace("/dc/", "")

            # Construct the absolute URL for the PDF file and add the "_1.pdf" suffix
            pdf_full_url = f"{base_url}{pdf_url.split('?')[0]}/_1.pdf"
            date_url = f"{base_url}{pdf_url.split('?')[0]}"
            date_response = requests.get(date_url)
            date_soup = BeautifulSoup(date_response.content, 'html.parser')
            date = date_soup.find('span', class_='dams-metadata-value', text='Date:').next_sibling.strip()

            # Create a dictionary with the PDF URL and date
            pdf_info = {
                'pdf_url': pdf_full_url,
                'date': datetime.strptime(date, '%Y-%m-%d').strftime('%Y_%m_%d')
            }


            # Append the dictionary to the list of PDF links
            pdf_links.append(pdf_info)
            print(f"PDF link found: {pdf_full_url}")
        except (TypeError, AttributeError):
            # Skip links that do not have a href attribute or have invalid HTML
            continue

# Write the PDF links to a JSON file
if len(pdf_links) > 0:
    with open('ucsd_pdf_links.json', 'w') as f:
        json.dump(pdf_links, f, indent=4)
else:
    print("No PDF links found.")


C:\Users\stacy\AppData\Local\Temp\ipykernel_34336\3112146848.py:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  date = date_soup.find('span', class_='dams-metadata-value', text='Date:').next_sibling.strip()


No PDF links found.
